<a href="https://colab.research.google.com/github/iris-aegis/PersonalAssistantOfficial/blob/main/%E5%85%AC%E9%96%8B%E7%94%A8orderSecRec%2Bwhisper%2BchatGPT%2BVOICEVOX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#対話生成部準備
#chatGPTのAPIの準備
!pip install openai --no-cache-dir
!pip install --upgrade pip
!pip install openai

import os
import openai
openai.api_key = ""#ここにchatGPTのAPIキーを入力してください

In [ ]:
#録音部準備
# from https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

In [ ]:
#音声認識部準備
#whisperのダウンロード
!pip install git+https://github.com/openai/whisper.git
!pip install sentencepiece
import whisper
model = whisper.load_model('medium')#colabがproで使用可能メモリが大きければlargeに変更可

参考サイト：https://monomonotech.jp/kurage/memo/230227_voicevox_colaboratory.html

In [ ]:
#音声合成部準備
#VOICEVOXのダウンロード
# 参考サイト：https://monomonotech.jp/kurage/memo/230227_voicevox_colaboratory.html

!curl -sSfL https://github.com/VOICEVOX/voicevox_core/releases/latest/download/download.sh | bash -s
%cd voicevox_core/
!wget https://github.com/VOICEVOX/voicevox_core/releases/download/0.14.1/voicevox_core-0.14.1+cuda-cp38-abi3-linux_x86_64.whl
!pip install voicevox_core-0.14.1+cuda-cp38-abi3-linux_x86_64.whl
!wget https://raw.githubusercontent.com/VOICEVOX/voicevox_core/main/example/python/run.py

In [ ]:
#音声再生部準備
# pyopenjtalkのインストール
!pip install pyopenjtalk
from IPython.display import Audio
from IPython.display import display

In [ ]:
#対話システム
inputfile = 'input.wav'#録音した音声ファイル名を指定
second = 3
counter = 1
while True:
  #録音部
  print(f"{counter}回目の会話：録音秒数を入力してください")
  counter ++
  second = int(input())#何秒録音するか指定する
  if record == 0 : break
  print(f"{second}秒間録音中です")
  record(second, inputfile)
  print("録音終了")

  #音声認識部
  audio = whisper.load_audio(inputfile)
  audio = whisper.pad_or_trim(audio)
  mel = whisper.log_mel_spectrogram(audio).to(model.device)
  _, probs = model.detect_language(mel)
  options = whisper.DecodingOptions()
  result = whisper.decode(model, mel, options)
  print(result.text)#認識結果のテキストを表示

  #対話生成部
  inputText=result.text#認識結果を対話生成部に渡す
  res = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system","content": "猫の口調メイドとして返答してください。"},
          #{"role": "assistant","content": log},#前回の対話を引き継ぎ
          {"role": "user","content": inputText},
      ],
  )
  print(res)#chatGPTのAPIの出力結果(消費したトークン数等)
  text = "、、、、、"+res["choices"][0]["message"]["content"]
  print(text)#生成した対話文のみ表示

  #音声合成部
  speaker_id = 14#46
  !sed -i "/SPEAKER_ID =/c SPEAKER_ID = $speaker_id" run.py
  !python ./run.py ./open_jtalk_dic_utf_8-1.11 $text output.wav

  #音声再生部
  wn = Audio("output.wav",rate = 22050,autoplay=True)
  display(wn)